### Resources:

* [Lets-Plot Usage Guide](https://nbviewer.org/github/JetBrains/lets-plot-kotlin/blob/master/docs/guide/user_guide.ipynb)
* [GitHub Repository](https://github.com/JetBrains/lets-plot-kotlin)

In [253]:
%use lets-plot

In [254]:
data class EvaluationMetrics(
    val evalNumber: String,
    val rating: Double,
    val ratingDeviation: Double,
    var avgFitness: Double = 0.0, // average over all runs
    var fitnessDeviation: Double = 0.0
)

class AlgorithmPerformance(val name: String) {
    val ratings: LinkedHashMap<String, EvaluationMetrics> = LinkedHashMap()

    fun addRating(evalNumber: String, rating: Double, ratingDeviation: Double) {
        ratings[evalNumber] = EvaluationMetrics(evalNumber, rating, ratingDeviation)
    }

    fun setAvgFitness(evalNumber: String, avgFitness: Double) {
        ratings[evalNumber]?.avgFitness = avgFitness
    }

    fun setFitnessDeviation(evalNumber: String, stdDeviation: Double) {
        ratings[evalNumber]?.fitnessDeviation = stdDeviation
    }
}

In [255]:
import org.um.feri.ears.algorithms.DummyAlgorithm
import org.um.feri.ears.algorithms.NumberAlgorithm
import org.um.feri.ears.benchmark.Benchmark
import org.um.feri.ears.examples.MovingPeaksBenchmark
import org.um.feri.ears.statistic.rating_system.RatingType
import org.um.feri.ears.visualization.rating.RatingIntervalPlot
import java.io.File

Benchmark.printInfo = false
DummyAlgorithm.readFromJson = false
val displayRatingChart = false

val algResultDir =
    "D:${File.separator}EDOLAB-MATLAB${File.separator}Results${File.separator}Comparison${File.separator}DeleteAfterTest"

val benchmark = "$algResultDir${File.separator}MPB_Peaks10_ChangeFrequency5000_D5_ShiftSeverity1_Environments3"

val players = arrayListOf<NumberAlgorithm>(
    DummyAlgorithm("DSPSO", benchmark),
    DummyAlgorithm("mQSO", benchmark),
    DummyAlgorithm("RPSO", benchmark),
    DummyAlgorithm("mjDE", benchmark),
    DummyAlgorithm("mPSO", benchmark)
)

// prepare objects for storing the algorithms data
val algPerf = players.map { AlgorithmPerformance(it.id) }.toMutableList()

val runNumber = 31 // number of runs

// calculate the number of different problems
val changeFrequency = 5000
val environmentNumber = 3
val sampleInterval = 1000
val numOfProblems = (changeFrequency / sampleInterval) * environmentNumber + environmentNumber - 1

var evaluationNumber = sampleInterval
for (i in 0 until numOfProblems) {
    // create and execute benchmarks for every evaluation defined with sampleInterval
    val movingPeaksBenchmark = MovingPeaksBenchmark(evaluationNumber).apply {
        setDisplayRatingCharts(false)
        addAlgorithms(players)
        run(runNumber)
    }

    val tournamentResults = movingPeaksBenchmark.tournamentResults

    if (displayRatingChart) {
        RatingIntervalPlot.displayChart(
            tournamentResults.players,
            RatingType.GLICKO2,
            "Rating Interval for Eval$evaluationNumber"
        )
    }

    // store tournament results for every player (algorithm)
    tournamentResults.players.forEach { player ->
        algPerf.firstOrNull { it.name == player.id }?.apply {
            addRating(
                "mpbeval$evaluationNumber",
                player.glicko2Rating.rating,
                player.glicko2Rating.ratingDeviation
            )
        }
    }

    if (evaluationNumber % changeFrequency == 0) {
        evaluationNumber++
    } else if (evaluationNumber % sampleInterval == 0) {
        evaluationNumber += sampleInterval
    } else {
        evaluationNumber = evaluationNumber + sampleInterval - 1
    }
}

TrueSkill One-On-One rating:
mPSO - Mean(mu)=4,03, Std-Dev(sigma)=1,09 RI=[1,86, 6,2]
mjDE - Mean(mu)=3,51, Std-Dev(sigma)=1,03 RI=[1,46, 5,56]
RPSO - Mean(mu)=2,62, Std-Dev(sigma)=1,04 RI=[0,54, 4,69]
mQSO - Mean(mu)=0,51, Std-Dev(sigma)=0,91 RI=[−1,31, 2,33]
DSPSO - Mean(mu)=−2,28, Std-Dev(sigma)=0,96 RI=[−4,19, −0,37]

TrueSkill Free-For-All rating:
mPSO - Mean(mu)=29,33, Std-Dev(sigma)=1,04 RI=[27,25, 31,41]
RPSO - Mean(mu)=28,11, Std-Dev(sigma)=1,03 RI=[26,06, 30,17]
mjDE - Mean(mu)=27,94, Std-Dev(sigma)=1,02 RI=[25,89, 29,99]
mQSO - Mean(mu)=22,36, Std-Dev(sigma)=1,03 RI=[20,29, 24,42]
DSPSO - Mean(mu)=19,06, Std-Dev(sigma)=1,15 RI=[16,76, 21,35]

Glicko2 rating:
mPSO - Rating=1.730,4 RD=50 ro=0,06 RI=[1630,43, 1830,43]
mjDE - Rating=1.672,8 RD=50 ro=0,06 RI=[1572,82, 1772,82]
RPSO - Rating=1.648,1 RD=50 ro=0,06 RI=[1548,13, 1748,13]
mQSO - Rating=1.310,7 RD=50 ro=0,06 RI=[1210,72, 1410,72]
DSPSO - Rating=1.137,9 RD=50 ro=0,06 RI=[1037,9, 1237,9]

Game results:
mPSO [win=90, lose

In [256]:
fun generateSequence(changeFrequency: Int, environmentNumber: Int, sampleInterval: Int = 1000): List<Int> {
    val result = mutableListOf<Int>()

    // Generate numbers increasing by 1000
    for (i in 1000..changeFrequency * environmentNumber step sampleInterval) {
        result.add(i)
        if (i % changeFrequency == 0 && i != changeFrequency * environmentNumber) {
            // Add the number just before the next multiple of 1000 (4999, 9999, etc.)
            result.add(i + 1)
        }
    }

    return result
}

In [265]:
import org.jetbrains.letsPlot.commons.values.Color

// number of algorithms in algPerf
val numAlgorithms = algPerf.size

// sample x-axis values (100 evaluations)
//val xValues = (1000..100000 step 1000).toList().subList(0, 100)
val xValues = generateSequence(changeFrequency, environmentNumber)

// find the global minimum and maximum y-values across all algorithms
//val allRatings = algPerf.flatMap { it.ratings.values.map { it.rating } }
//val minY = allRatings.minOrNull() ?: 0.0
//val maxY = allRatings.maxOrNull() ?: 1.0

// create the base plot (without any layers yet)
var plot = letsPlot(mapOf("eval" to xValues))

// loop through all algorithms in algPerf
for (i in 0 until numAlgorithms) {
    // extract the rating values for the current algorithm
    val yValues = algPerf[i].ratings.values.map { it.rating }

    // calculate the lower and upper bounds for the rating deviations
    val lowerBounds = algPerf[i].ratings.values.map { it.rating - it.ratingDeviation }
    val upperBounds = algPerf[i].ratings.values.map { it.rating + it.ratingDeviation }

    // prepare data for this algorithm
    val data = mapOf(
        "eval" to xValues,
        "rating" to yValues,
        "lowerBounds" to lowerBounds,
        "upperBounds" to upperBounds,
        "Algorithm" to List(numOfProblems) { algPerf[i].name }  // label each algorithm
    )

    // add rating line to the plot (cycling through predefined colors)
    plot += geomLine(data = data) {
        x = "eval"
        y = "rating"
        color = "Algorithm"
    }

    // add rating deviation ribbon to the plot (using the same color with transparency)
    plot += geomRibbon(data = data, alpha = 0.2) {
        x = "eval"
        ymin = "lowerBounds"
        ymax = "upperBounds"
        color = "Algorithm"
        fill = "Algorithm"
    }

    // add points at the same locations as the line
    plot += geomPoint(data = data) {
        x = "eval"
        y = "rating"
        color = "Algorithm"
    }
}

// add vertical lines at every 5000th eval point
/*val verticalLines = (changeFrequency..changeFrequency * environmentNumber step changeFrequency).toList()
verticalLines.forEach { evalPoint ->
    plot += geomVLine(xintercept = evalPoint, color = "grey", linetype = "dashed") {
        // You can customize the appearance here
    }
}*/

// set y-axis limits to min and max y-values
//
plot += scaleYContinuous(limits = 1000 to 2000)
plot += scaleXContinuous(breaks = xValues, labels = xValues.map { it.toInt().toString() })  // specify the tick marks
//plot += scaleYContinuous(limits = minY to maxY)
//plot += scaleXContinuous(limits = 1000 to 100000)

// set plot size and title
plot += ggsize(1500, 750)
plot += ggtitle("DOAs ratings during optimization")

// display the plot
plot.show()

// save the plot
ggsave(plot, "plot.svg")

D:\EARS\src\org\um\feri\ears\examples\lets-plot-images\plot.svg

In [261]:
import org.jetbrains.letsPlot.letsPlot
import org.jetbrains.letsPlot.ggsize
import org.jetbrains.letsPlot.geom.geomLine
import org.jetbrains.letsPlot.geom.geomRibbon
import org.jetbrains.letsPlot.geom.geomPoint
import org.jetbrains.letsPlot.scale.scaleXContinuous
import org.jetbrains.letsPlot.scale.scaleYContinuous

// specify the eval numbers you're interested in
val xZoomed = listOf(5000, 5001)

// lists to store the combined data for all algorithms
val allEvalValues = mutableListOf<Int>()
val allRatingValues = mutableListOf<Double>()
val allLowerBounds = mutableListOf<Double>()
val allUpperBounds = mutableListOf<Double>()
val allAlgorithms = mutableListOf<String>()

// loop through all algorithms and gather the data into the combined lists
for (i in 0 until numAlgorithms) {
    // filter ratings to include only the eval numbers 5000, 5001
    val ratings = algPerf[i].ratings.values
        .filter { eval -> eval.evalNumber.substring(7, 11).toInt() in xZoomed }

    val yZoomed = ratings.map { it.rating }
    val lowerBoundsZoomed = ratings.map { it.rating - it.ratingDeviation }
    val upperBoundsZoomed = ratings.map { it.rating + it.ratingDeviation }

    // append the data to the combined lists
    allEvalValues.addAll(ratings.map { it.evalNumber.substring(7, 11).toInt() })
    allRatingValues.addAll(yZoomed)
    allLowerBounds.addAll(lowerBoundsZoomed)
    allUpperBounds.addAll(upperBoundsZoomed)
    allAlgorithms.addAll(List(yZoomed.size) { algPerf[i].name })
}

// prepare the combined data for the plot
val combinedData = mapOf(
    "eval" to allEvalValues,
    "rating" to allRatingValues,
    "lowerBounds" to allLowerBounds,
    "upperBounds" to allUpperBounds,
    "Algorithm" to allAlgorithms
)

// create a single plot with all algorithms
val combinedPlot = letsPlot(combinedData) +
        geomLine {
            x = "eval"
            y = "rating"
            color = "Algorithm"
        } +
        geomRibbon(alpha = 0.2) {
            x = "eval"
            ymin = "lowerBounds"
            ymax = "upperBounds"
            color = "Algorithm"
            fill = "Algorithm"
        } +
        geomPoint {
            x = "eval"
            y = "rating"
            color = "Algorithm"
        } +
        scaleXContinuous(breaks = listOf(5000, 5001), labels = listOf("5000", "5001")) +  // specify the tick marks, format labels as integers
        scaleYContinuous(limits = 1000 to 2000) +  // adjust y-axis limits if needed
        ggsize(1500, 750) +  // set plot size
        ggtitle("DOAs ratings just before and just after the change")

// display the combined plot
combinedPlot.show()

// save the combined plot
ggsave(combinedPlot, "plot_change.svg")

D:\EARS\src\org\um\feri\ears\examples\lets-plot-images\plot_change.svg